In [2]:
import pandas as pd
import numpy as np
import os.path
import os
from random import *

In [23]:
df=pd.read_parquet('data/ratings.gzip')
df.head()

,userId,movieId,itemId
0,1,0132793040,5.0
1,1,B005UEB5TQ,5.0
2,2,0321732944,5.0
3,2,B000W3LJ6Y,5.0
4,2,B001EYU79A,5.0


In [24]:
ar=df.to_numpy()

In [25]:
len(df)

7824482

In [27]:
def inteName2(i):
    if i==0:
        return "1-9"
    elif i<10:
        return str(int(i*10))+"-"+str(int(i*10+9))
    elif i>=20:
        return 'sup200'
    else :
        return str(int(i*10))+"-"+str(int(i*10+49))

In [3]:
def inteName(i):
    if i==0:
        return "1-9"
    else :
        return str(int(i*10))+"-"+str(int(i*10+9))

In [29]:
def nextUser(ar,l):
    userID=ar[l,0]
    t=len(ar)
    nxt=l+1
    while((nxt<t)and(ar[nxt,0]==userID)):
        nxt+=1
    return nxt-1

In [30]:
def listInteTestInf10(l):
    listInte=list()
    size=len(l)
    cpt=0
    
    if size ==10:
        for i in range(size):
            listInte.append(l[i])
            shuffle(listInte)
        return listInte
    
    while (cpt+size<10) :
        for i in range(size):
            listInte.append(l[i])
            cpt+=1
    if cpt!=10:
        for i in range(10-cpt):
            listInte.append(l[randint(0, size-1)])
    shuffle(listInte)
    return listInte

In [31]:
def listInteTestSup10(l):
    listInte=list()
    size=len(l)
    nbLine=size//5
    for turn in range(2):
        l2=l.copy()
        shuffle(l2)
        for i in range(4):
            
            lTemp=list()
            for k in range(nbLine):
                lTemp.append(l2[0])
                del l2[0]
            listInte.append(lTemp)
        listInte.append(l2)
    return listInte

In [32]:
def listAppend(start,end):
    sizeInte=end-start+1
    inte=list()
    for i in range(start,end+1):
        inte.append(i)
    
    
    if sizeInte<=10 :
        lTest=listInteTestInf10(inte)
        for fold in range(10) :
            dicoNb[fold][ar[start,0]]=(sizeInte-1,1)
            t=lTest[fold]
            for l in inte:
                if l==t :
                    dicoTest[fold].append(l)
                else :
                    dicoTrain[fold].append(l)
    
    else :
        lTest=listInteTestSup10(inte)
        for fold in range(10) :
            t=lTest[fold]
            dicoNb[fold][ar[start,0]]=(sizeInte-len(t),len(t))
            for l in inte :
                if l in t :
                    dicoTest[fold].append(l)
                else :
                    dicoTrain[fold].append(l)

In [33]:
lastLine=len(df)-1
start=0
end=nextUser(ar,start)
dicoTrain={}
dicoTest={}
dicoNb={}
listOne=list()
for fold in range(10):
    dicoNb[fold]={}
    dicoTrain[fold]=list()
    dicoTest[fold]=list()
    
while (end!=lastLine):
    if end!=start:
        listAppend(start,end)
    else :
        listOne.append([ar[start]])
    start=end+1
    end=nextUser(ar,start)
if end!=start:
    listAppend(start,end)

In [34]:
arOne=np.concatenate(listOne)
dfOne=pd.DataFrame({'userId':arOne[:,0],'movieId':arOne[:,1],'rating':arOne[:,2]})
dfOne.to_parquet('dataCV3/dfOne.gzip', compression='gzip',index=False)
print(len(dfOne))
dfOne.head()

2881831


,userId,movieId,rating
0,3,0439886341,1.0
1,4,0439886341,3.0
2,5,0439886341,1.0
3,10,0511189877,5.0
4,12,0528881469,5.0


In [35]:
del listOne,arOne,dfOne

In [36]:
for fold in range(10):
    print("--== save nb ratings ==-- ")
    ID=list(dicoNb[fold].keys())
    nbTrain=list()
    nbTest=list()
    for i in ID:
        (a,b)=dicoNb[fold][i]
        nbTrain.append(a)
        nbTest.append(b)
    dfDico = pd.DataFrame({'userId' : ID, 'NBRatingTrain' : nbTrain,'NBRatingTest' : nbTest})
    dfDico.to_csv('dataCV3/user-nbRating_fold_'+str(fold)+'.csv',index =False)
    del ID,dfDico
del dicoNb

--== save nb ratings ==-- 
--== save nb ratings ==-- 
--== save nb ratings ==-- 
--== save nb ratings ==-- 
--== save nb ratings ==-- 
--== save nb ratings ==-- 
--== save nb ratings ==-- 
--== save nb ratings ==-- 
--== save nb ratings ==-- 
--== save nb ratings ==-- 


In [37]:
for fold in range(10):
    ldf=list()
    for l in dicoTest[fold]:
        ldf.append([ar[l]])
    arTest=np.concatenate(ldf)
    dfTest=pd.DataFrame({'userId':arTest[:,0],'movieId':arTest[:,1],'rating':arTest[:,2]})
    dfTest.to_parquet('dataCV3/dfTest_fold_'+str(fold)+'.gzip', compression='gzip',index=False)
    del arTest,dfTest
    
    ldf=list()
    for l in dicoTrain[fold]:
        ldf.append([ar[l]])
    arTrain=np.concatenate(ldf)
    dfTrain=pd.DataFrame({'userId':arTrain[:,0],'movieId':arTrain[:,1],'rating':arTrain[:,2]})
    dfTrain.to_parquet('dataCV3/dfTrain_fold_'+str(fold)+'.gzip', compression='gzip',index=False)
    del arTrain,dfTrain
del ldf

In [38]:
del dicoTrain,dicoTest

In [39]:
for fold in range(10):
    print("______________________________________________________________")
    print("fold : ",fold)
    print("load nb ratings")
    arDico=pd.read_csv('dataCV3/user-nbRating_fold_'+str(fold)+'.csv').to_numpy()
    d={}
    for i in range(len(arDico)) :
        d[arDico[i,0]]=arDico[i,1]
    del arDico
    
    print("load testing dataset")
    arTest=pd.read_parquet('dataCV3/dfTest_fold_'+str(fold)+'.gzip').to_numpy()
    size=len(arTest)
    dicoDf={}
    print("Repartition Start")
    for j in range(size): 
        #print(arTest[j,0],d[int(arTest[j,0])])
        nb=d[int(arTest[j,0])]//10
        if nb in dicoDf :
            dicoDf[nb].append([arTest[j]])
        else:
            dicoDf[nb]=list()
            dicoDf[nb].append([arTest[j]])

    print("Sauvegarde des intervalles")
    for j in dicoDf:
        a=np.concatenate(dicoDf[j],axis=0)
        dfTemp = pd.DataFrame({'userId':a[:,0],'movieId':a[:,1],'rating':a[:,2]})
        dfTemp.to_parquet('dataCV3/df'+inteName(j)+'_fold_'+str(fold)+'.gzip', compression='gzip',index=False)
        del dfTemp,a
    
    print("Creation of the large intervalles")
    for i in range(10,20,5):
        listDF=list()
        for j in range(5):
            path='dataCV3/df'+inteName(i)+'_fold_'+str(fold)+'.gzip'
            if os.path.exists(path):
                listDF.append(pd.read_parquet(path))
        if len(listDF) != 0:
            dfTemp=pd.concat(listDF)
            dfTemp.to_parquet('dataCV3/df'+inteName2(i)+'_fold_'+str(fold)+'.gzip', compression='gzip',index=False)
    
    print("Creation of the intervalle sup500")
    listDF=list()
    for i in range(20,800):
        path='dataCV3/df'+inteName(i)+'_fold_'+str(fold)+'.gzip'
        if os.path.exists(path):
            listDF.append(pd.read_parquet(path))
    if len(listDF) != 0:
        dfTemp=pd.concat(listDF)
        dfTemp.to_parquet('dataCV3/df'+inteName2(50)+'_fold_'+str(fold)+'.gzip', compression='gzip',index=False)
    del dfTemp,listDF
    
    print("Creation of the intervalle inf20")
    df1=pd.read_parquet('dataCV3/df1-9'+'_fold_'+str(fold)+'.gzip')
    df2=pd.read_parquet('dataCV3/df10-19'+'_fold_'+str(fold)+'.gzip')
    dfInf20=pd.concat([df1,df2])
    dfInf20.to_parquet('dataCV3/dfInf20'+'_fold_'+str(fold)+'.gzip', compression='gzip',index=False)
    del df1,df2,dfInf20
    
    print("Creation of the intervalle sup20")
    listDF=list()
    for i in range(2,10):
        listDF.append(pd.read_parquet('dataCV3/df'+inteName2(i)+'_fold_'+str(fold)+'.gzip'))
    for i in range(10,51,5):
        path='dataCV3/df'+inteName(i)+'_fold_'+str(fold)+'.gzip'
        if os.path.exists(path):
            listDF.append(pd.read_parquet(path))
    dfSup20=pd.concat(listDF)
    dfSup20.to_parquet('dataCV3/dfSup20_fold_'+str(fold)+'.gzip', compression='gzip',index=False)    

______________________________________________________________
fold :  0
load nb ratings
load testing dataset
Repartition Start
Sauvegarde des intervalles
Creation of the large intervalles
Creation of the intervalle sup500
Creation of the intervalle inf20
Creation of the intervalle sup20
______________________________________________________________
fold :  1
load nb ratings
load testing dataset
Repartition Start
Sauvegarde des intervalles
Creation of the large intervalles
Creation of the intervalle sup500
Creation of the intervalle inf20
Creation of the intervalle sup20
______________________________________________________________
fold :  2
load nb ratings
load testing dataset
Repartition Start
Sauvegarde des intervalles
Creation of the large intervalles
Creation of the intervalle sup500
Creation of the intervalle inf20
Creation of the intervalle sup20
______________________________________________________________
fold :  3
load nb ratings
load testing dataset
Repartition Start
Sauve

In [4]:
for fold in range(10):
    print('- ',fold,' -')
    for i in range(10,5000):
        path='dataCV3/df'+inteName(i)+'_fold_'+str(fold)+'.gzip'
        if os.path.exists(path):
            os.remove(path)

-  0  -
-  1  -
-  2  -
-  3  -
-  4  -
-  5  -
-  6  -
-  7  -
-  8  -
-  9  -
